In [141]:
# important, you need to install scvelo

import os
import numpy as np
import pandas as pd
import scanpy as sc
import scvelo as scv
import matplotlib.pyplot as plt
import magic
from scipy.stats import spearmanr

In [142]:
path_cwd=os.getcwd()
path_train_data=path_cwd + '/ML4G_Project_2_Data/train_data'
path_test_data=path_cwd + '/ML4G_Project_2_Data/test_data'

In [143]:
#train_data
bulk_train=pd.read_csv(path_train_data +'/pancreas_bulk_train.csv',index_col=0)
metadata_train=pd.read_csv(path_train_data+ '/pancreas_sc_metadata_train.csv')
sc_train=pd.read_csv(path_train_data +'/pancreas_sc_train.csv', index_col=0)

#test_data
metadata_test=pd.read_csv(path_test_data+ '/pancreas_sc_metadata_test_wocelltype.csv')
sc_test=pd.read_csv(path_test_data +'/pancreas_sc_test.csv', index_col=0)

In [144]:
cells_1=[column for column in sc_train.columns if 'patient1' in column ]
cells_2=[column for column in sc_train.columns if 'patient2' in column ]
cells_3=[column for column in sc_train.columns if 'patient3' in column ]
cells_4=[column for column in sc_train.columns if 'patient4' in column ]

# For the moment we work only with the first patient
sc1_train=sc_train[cells_1].T # we transpose the dataframe since we want a cells x genes dataframe for applying MAGIC
sc1_train.head(3)

Unnamed: 0,SGIP1,AZIN2,CLIC4,AGBL4,NECAP2,SLC45A1,TGFBR3,DBT,RFWD2,C1orf21,...,LOC389831,MGC70870,KIR2DS5,KIR2DL5A,KIR3DS1,KIR2DL5B,KIR2DS2,KIR2DS1,KIR2DL2,KIR2DS3
patient1_A10,0.0,0.0,127.0,0.0,140.0,0.0,0.0,1.0,103.0,267.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
patient1_A12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
patient1_A13,0.0,110.0,0.0,21.0,0.0,0.0,0.0,56.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [145]:
# Pre processing: Normalize per total reads per cell and transform with log
sc1_train=ad.AnnData(sc1_train) #Needed to use scv functions for pre processing
scv.pp.normalize_per_cell(sc1_train)
scv.pp.log1p(sc1_train)
sc1_train=pd.DataFrame(sc1_train.X, columns = sc1_train.var_names,index=sc1_train.obs_names)

sc1_train.head(3)

Normalized count data: X.


Unnamed: 0,SGIP1,AZIN2,CLIC4,AGBL4,NECAP2,SLC45A1,TGFBR3,DBT,RFWD2,C1orf21,...,LOC389831,MGC70870,KIR2DS5,KIR2DL5A,KIR3DS1,KIR2DL5B,KIR2DS2,KIR2DS1,KIR2DL2,KIR2DS3
patient1_A10,0.0,0.000000,4.584115,0.000000,4.680622,0.0,0.0,0.567086,4.377034,5.321808,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
patient1_A12,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,3.694045,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
patient1_A13,0.0,4.803261,0.000000,3.181477,0.000000,0.0,0.0,4.136011,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [147]:
from itertools import product

results=[]
k_set, t_set , n_pca_set= [2,4,6,8,10], [2,3,4,6,8], [100,200,300]
for k,t, n_pca in product(k_set,t_set,n_pca_set):
    magic_op = magic.MAGIC(verbose=0,knn=k,t=t,n_pca=n_pca)
    imputed = magic_op.fit_transform(sc1_train)
    pred=imputed.mean(axis=0)
    bulk=bulk_train['patient1']
    results+=[spearmanr(pred,bulk).statistic]

C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:586: DeprecationWarning: Call to deprecated method warning. (Use TaskLogger.log_warning instead) -- Deprecated since version 1.1.0.
  _logger.warning(
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:748: DeprecationWarning: Call to deprecated method task. (Use TaskLogger.log_task instead) -- Deprecated since version 1.1.0.
  with _logger.task("imputation"):
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:650: DeprecationWarning: Call to deprecated method task. (Use TaskLogger.log_task instead) -- Deprecated since version 1.1.0.
  with _logger.task("MAGIC"):
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:381: DeprecationWarning: Call to deprecated method info. (Use TaskLogger.log_info instead) -- Deprecated since version 1.1.0.
  _logger.info(
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:436: 

  Running MAGIC with `solver='exact'` on 25453-dimensional data may take a long time. Consider denoising specific genes with `genes=<list-like>` or using `solver='approximate'`.


C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:586: DeprecationWarning: Call to deprecated method warning. (Use TaskLogger.log_warning instead) -- Deprecated since version 1.1.0.
  _logger.warning(
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:748: DeprecationWarning: Call to deprecated method task. (Use TaskLogger.log_task instead) -- Deprecated since version 1.1.0.
  with _logger.task("imputation"):
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:650: DeprecationWarning: Call to deprecated method task. (Use TaskLogger.log_task instead) -- Deprecated since version 1.1.0.
  with _logger.task("MAGIC"):
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:381: DeprecationWarning: Call to deprecated method info. (Use TaskLogger.log_info instead) -- Deprecated since version 1.1.0.
  _logger.info(
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:436: 

  Running MAGIC with `solver='exact'` on 25453-dimensional data may take a long time. Consider denoising specific genes with `genes=<list-like>` or using `solver='approximate'`.


C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:586: DeprecationWarning: Call to deprecated method warning. (Use TaskLogger.log_warning instead) -- Deprecated since version 1.1.0.
  _logger.warning(
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:748: DeprecationWarning: Call to deprecated method task. (Use TaskLogger.log_task instead) -- Deprecated since version 1.1.0.
  with _logger.task("imputation"):
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:650: DeprecationWarning: Call to deprecated method task. (Use TaskLogger.log_task instead) -- Deprecated since version 1.1.0.
  with _logger.task("MAGIC"):
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:381: DeprecationWarning: Call to deprecated method info. (Use TaskLogger.log_info instead) -- Deprecated since version 1.1.0.
  _logger.info(
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:436: 

...

C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:586: DeprecationWarning: Call to deprecated method warning. (Use TaskLogger.log_warning instead) -- Deprecated since version 1.1.0.
  _logger.warning(
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:748: DeprecationWarning: Call to deprecated method task. (Use TaskLogger.log_task instead) -- Deprecated since version 1.1.0.
  with _logger.task("imputation"):
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:650: DeprecationWarning: Call to deprecated method task. (Use TaskLogger.log_task instead) -- Deprecated since version 1.1.0.
  with _logger.task("MAGIC"):
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:381: DeprecationWarning: Call to deprecated method info. (Use TaskLogger.log_info instead) -- Deprecated since version 1.1.0.
  _logger.info(
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:436: 

  Running MAGIC with `solver='exact'` on 25453-dimensional data may take a long time. Consider denoising specific genes with `genes=<list-like>` or using `solver='approximate'`.


C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:586: DeprecationWarning: Call to deprecated method warning. (Use TaskLogger.log_warning instead) -- Deprecated since version 1.1.0.
  _logger.warning(
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:748: DeprecationWarning: Call to deprecated method task. (Use TaskLogger.log_task instead) -- Deprecated since version 1.1.0.
  with _logger.task("imputation"):
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:650: DeprecationWarning: Call to deprecated method task. (Use TaskLogger.log_task instead) -- Deprecated since version 1.1.0.
  with _logger.task("MAGIC"):
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:381: DeprecationWarning: Call to deprecated method info. (Use TaskLogger.log_info instead) -- Deprecated since version 1.1.0.
  _logger.info(
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:436: 

  Running MAGIC with `solver='exact'` on 25453-dimensional data may take a long time. Consider denoising specific genes with `genes=<list-like>` or using `solver='approximate'`.


C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:586: DeprecationWarning: Call to deprecated method warning. (Use TaskLogger.log_warning instead) -- Deprecated since version 1.1.0.
  _logger.warning(
C:\Users\gugli\AppData\Roaming\Python\Python310\site-packages\magic\magic.py:748: DeprecationWarning: Call to deprecated method task. (Use TaskLogger.log_task instead) -- Deprecated since version 1.1.0.
  with _logger.task("imputation"):


In [149]:
baseline=spearmanr(sc1_train.mean(axis=0),bulk_train['patient1']).statistic
print(f'Correlation before imputation: {baseline}')
print(f'Best Correlation: {max(results)}')

Correlation before imputation: 0.9232437448988214
Best Correlation: 0.9216702939167969


# Draft

In [ ]:
# To do: HPO for MAGIC
from sklearn.base import BaseEstimator

# CREATION OF SCORE FUNCTION
def score_func(y, y_pred):
    return spearmanr(y,y_pred).statistic

scorer=make_scorer(score_func) #needed to be able to use spearmanr as score function in scikit-learn

class custom_magic(BaseEstimator):
    def __init__(self,k=5,t=3,n_pca=100):
        self.k=k
        self.t=t
        self.n_pca=n_pca
        self.model=magic.MAGIC(verbose=0,knn=k,t=t,n_pca=n_pca)

    def fit(self, X, y=None):
        return None

    def predict(self, X):
        raw=self.model.fit_transform(X)
        return raw.mean(axis=0)


    def set_params(self, **params):
        if not params:
            return self

        for key, value in params.items():
            if hasattr(self, key):
                setattr(self, key, value)
            else:
                self.kwargs[key] = value
        return self

param_grid= {'k':[1,2,3,5],'t':[1,2,3], 'n_pca':[10,20,30]}

from sklearn.model_selection import GridSearchCV

GridSearchCV(custom_magic(), param_grid,cv=,scoring=score_func())